In [1]:
#pragma cling add_include_path("../../libtorch/include")
#pragma cling add_include_path("../../libtorch/include/torch/csrc/api/include")
#pragma cling add_library_path("../../libtorch/lib")
#pragma cling load("libtorch")

In [2]:
#include <iostream>
#include <tuple>
#include <string>
#include <vector>
#include <torch/torch.h>
#include <torch/script.h>
namespace nn = torch::nn;

In [3]:
// for Reproducibility, set random seed. the randomness is same with python version
int m_seed = 11;
bool m_torch_deterministic = true;
srand(m_seed);
torch::manual_seed(m_seed);
at::globalContext().setDeterministicCuDNN(m_torch_deterministic ? true : false);
// https://docs.nvidia.com/cuda/cublas/index.html#cublasApi_reproducibility
at::globalContext().setDeterministicAlgorithms(m_torch_deterministic ? true : false, true);

In [4]:
std::cout << torch::randn({3,5}) << std::endl;

 0.7376  1.9459 -0.6995 -1.3023 -0.5133
-0.2696  0.2462  0.4839  0.4504 -0.9568
 1.5012 -0.3136 -0.2343 -1.0713  0.1648
[ CPUFloatType{3,5} ]


In [5]:
torch::Tensor new_x = torch::randn({4,3});
std::cout << new_x << std::endl;

 0.1157  0.3834 -0.1234
 0.8929 -0.4891 -1.4470
 1.1941  0.5462 -2.0750
-0.9691 -2.1354 -1.0431
[ CPUFloatType{4,3} ]


In [6]:
std::cout << torch::cuda::is_available() << std::endl;

1


In [7]:
std::vector<char> get_the_bytes(std::string filename) {
    std::ifstream input(filename, std::ios::binary);
    std::vector<char> bytes(
        (std::istreambuf_iterator<char>(input)),
        (std::istreambuf_iterator<char>()));

    input.close();
    return bytes;
}

In [8]:
std::string pt_path = "./none_embeded_model_state_dict.pt";
std::vector<char> f = get_the_bytes(pt_path);

In [9]:
void print_vector(const std::vector<char> v){
    for(auto x:v){
        std::cout << x;
    }
}

In [10]:
print_vector(f);

In [11]:
c10::Dict<torch::jit::IValue, torch::jit::IValue> weights = torch::pickle_load(f).toGenericDict();

In [12]:
torch::NoGradGuard no_grad;
for (auto const& w : weights) {
    std::string name = w.key().toStringRef();
    at::Tensor param = w.value().toTensor();
    std::cout << name << std::endl;
    std::cout << param << std::endl;
}

~~~
class MyModule(torch.nn.Module):
    def __init__(self,N, M):
        super(MyModule, self).__init__()
        self.linear1 = torch.nn.Linear(N, M)
        self.linear2 = torch.nn.Linear(M, 1)

    def forward(self, input):
        out0 = self.linear1(input)
        out0_relu = torch.nn.functional.relu(out0)
        return self.linear2(out0_relu)
~~~

In [13]:
class MyModuleImpl : public torch::nn::Module {
    public:
    MyModuleImpl(int64_t N, int64_t M) {
    linear1 = register_module("linear1", torch::nn::Linear(N, M));
    linear2 = register_module("linear2", torch::nn::Linear(M,1));
    //another_bias = register_parameter("b", torch::randn(M));
  }
    
  torch::Tensor forward(torch::Tensor input) {
    torch::Tensor out0 = linear1(input);
    torch::Tensor out0_relu = torch::nn::functional::relu(out0);  
    return linear2(out0_relu);
  }
    
  torch::nn::Linear linear1{nullptr}, linear2{nullptr};
  //torch::Tensor another_bias;
};
TORCH_MODULE(MyModule);

In [14]:
MyModule net{2,3};
torch::OrderedDict<std::string, torch::Tensor> ordered_parameter_dict = net->named_parameters();
for (const auto& pair : ordered_parameter_dict) {
  std::cout << pair.key() << ": " << pair.value() << std::endl;
}

In [15]:
void init_weights(nn::Module& module) {
	torch::NoGradGuard noGrad;
	if (auto* linear = module.as<torch::nn::Linear>()) {
        nn::init::ones_(linear->weight);
        nn::init::zeros_(linear->bias);
	}
}

In [16]:
net->apply(init_weights);

In [17]:
ordered_parameter_dict = net->named_parameters();
for (const auto& pair : ordered_parameter_dict) {
  std::cout << pair.key() << ": " << pair.value() << std::endl;
}

## none-embeded libtorch model load pretrained params from pytorch

In [18]:
class PretrainedModel : public torch::nn::Module {
    public:
    torch::nn::Linear linear1{nullptr}, linear2{nullptr};
    
    public:
    
    PretrainedModel(int64_t N, int64_t M) {
        linear1 = register_module("linear1", torch::nn::Linear(N, M));
        linear2 = register_module("linear2", torch::nn::Linear(M,1));
    }
    
    
    void load_parameters(std::string pt_path){
        std::vector<char> f = get_the_bytes(pt_path);
        c10::Dict<torch::jit::IValue, torch::jit::IValue> weights = torch::pickle_load(f).toGenericDict();
        const torch::OrderedDict<std::string, at::Tensor>& model_params = this->named_parameters();
        
    
        std::vector<std::string> param_names;
        for (auto const& w : model_params) {
            //std::cout << w.key() << std::endl;
            param_names.push_back(w.key());
          }

        torch::NoGradGuard no_grad;
        for (auto const& w : weights) {
            std::string name = w.key().toStringRef();
            at::Tensor param = w.value().toTensor();
            //std::cout << name << std::endl;
            //std::cout <<param << std::endl;

            if (std::find(param_names.begin(), param_names.end(), name) != param_names.end()){
                model_params.find(name)->copy_(param);
            } else {
                std::cout << name << " does not exist among model parameters." << std::endl;
            };
        }
    }
    
    torch::Tensor forward(torch::Tensor input) {
        torch::Tensor out0 = linear1(input);
        torch::Tensor out0_relu = torch::nn::functional::relu(out0);  
        return linear2(out0_relu);
    }
    
  //torch::Tensor another_bias;
};

In [19]:
PretrainedModel pretrained_model{2,3};

In [20]:
for (const auto& pair : pretrained_model.named_parameters()) {
  std::cout << pair.key() << ": " << pair.value() << std::endl;
}

In [21]:
pretrained_model.apply(init_weights);

In [22]:
for (const auto& pair : pretrained_model.named_parameters()) {
  std::cout << pair.key() << ": " << pair.value() << std::endl;
}

In [23]:
pretrained_model.load_parameters(pt_path);

In [24]:
for (const auto& pair : pretrained_model.named_parameters()) {
  std::cout << pair.key() << ": " << pair.value() << std::endl;
}

In [25]:
torch::Tensor x = torch::ones({4,2});
std::cout << x << std::endl;

In [26]:
torch::Tensor real_output = pretrained_model.forward(x);
std::cout << real_output <<std::endl;

# embeded model architecture

~~~
class EmbedModule(torch.nn.Module):
    def __init__(self):
        super(EmbedModule, self).__init__()
        self.linear0 = torch.nn.Linear(3, 2)
        self.model = MyModule(2,3)
        
    def forward(self, input):
        output1 = self.linear0(input)
        output2 = self.model(output1)
        return output2
~~~

In [27]:
class EmbededModuleImpl : public torch::nn::Module{
    public:
    torch::nn::Linear linear0{nullptr};
    MyModule model{nullptr};
    
    public:
    EmbededModuleImpl(int64_t P, int64_t N, int64_t M){
        linear0 = register_module("linear0", torch::nn::Linear(P,N));
        model = register_module("model", MyModule(N, M));
        
        
    }
    
    torch::Tensor forward(torch::Tensor x){
        torch::Tensor output1 = linear0(x);
        torch::Tensor output2 = model(output1);
        return output2;
    }
    
};
TORCH_MODULE(EmbededModule);

In [28]:
EmbededModule EmbededModule_ptr{3,2,3};

In [29]:
for (const auto& pair : EmbededModule_ptr->named_parameters()) {
  std::cout << pair.key() << ": " << pair.value() << std::endl;
}

## 2.2 embeded libtorch model load pretrained params from pytorch

In [30]:
class EmbededPretrainedModelImpl : public torch::nn::Module {
    
    public:
    torch::nn::Linear linear0{nullptr};
    MyModule model{nullptr};
    
    
    public:
    
    EmbededPretrainedModelImpl(int64_t P, int64_t N, int64_t M){
        linear0 = register_module("linear0", torch::nn::Linear(P,N));
        model = register_module("model", MyModule(N, M));
        
        
    }
    
    torch::Tensor forward(torch::Tensor x){
        torch::Tensor output1 = linear0(x);
        torch::Tensor output2 = model(output1);
        return output2;
    }
    
    
    void load_parameters(std::string pt_path){
        std::vector<char> f = get_the_bytes(pt_path);
        c10::Dict<torch::jit::IValue, torch::jit::IValue> weights = torch::pickle_load(f).toGenericDict();
        const torch::OrderedDict<std::string, at::Tensor>& model_params = this->named_parameters();
        
    
        std::vector<std::string> param_names;
        for (auto const& w : model_params) {
            //std::cout << w.key() << std::endl;
            param_names.push_back(w.key());
          }

        torch::NoGradGuard no_grad;
        for (auto const& w : weights) {
            std::string name = w.key().toStringRef();
            at::Tensor param = w.value().toTensor();
            //std::cout << name << std::endl;
            //std::cout <<param << std::endl;

            if (std::find(param_names.begin(), param_names.end(), name) != param_names.end()){
                model_params.find(name)->copy_(param);
            } else {
                std::cout << name << " does not exist among model parameters." << std::endl;
            };
        }
    }
   
  //torch::Tensor another_bias;
};
TORCH_MODULE(EmbededPretrainedModel);

In [31]:
EmbededPretrainedModel EmbededPretrainedModel_ptr{3,2,3};

In [32]:
for (const auto& pair : EmbededPretrainedModel_ptr->named_parameters()) {
  std::cout << pair.key() << ": " << pair.value() << std::endl;
}

In [33]:
EmbededPretrainedModel_ptr->apply(init_weights);

In [34]:
for (const auto& pair : EmbededPretrainedModel_ptr->named_parameters()) {
  std::cout << pair.key() << ": " << pair.value() << std::endl;
}

In [35]:
std::string embeded_pt_path = "./embeded_model_state_dict.pt";
EmbededPretrainedModel_ptr->load_parameters(embeded_pt_path);

In [36]:
for (const auto& pair : EmbededPretrainedModel_ptr->named_parameters()) {
  std::cout << pair.key() << ": " << pair.value() << std::endl;
}

In [37]:
torch::Tensor output = EmbededPretrainedModel_ptr(new_x);
std::cout << output << std::endl;